In [1]:
!pip install split-folders

  Using cached split_folders-0.5.1-py3-none-any.whl (8.4 kB)


In [2]:
!pip install tensorflow

  Using cached numpy-1.24.3-cp39-cp39-win_amd64.whl (14.9 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.25.2
    Uninstalling numpy-1.25.2:
      Successfully uninstalled numpy-1.25.2


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
daal4py 2021.5.0 requires daal==2021.4.0, which is not installed.
numba 0.55.1 requires numpy<1.22,>=1.18, but you have numpy 1.24.3 which is incompatible.
bokeh 2.4.2 requires typing-extensions>=3.10.0, but you have typing-extensions 3.7.4.3 which is incompatible.


In [30]:
!pip install -U tensorflow

In [31]:
!pip install keras

In [32]:
!python -m pip show tensorflow

Name: tensorflow
Version: 2.13.0
Summary: TensorFlow is an open source machine learning framework for everyone.
Home-page: https://www.tensorflow.org/
Author: Google Inc.
Author-email: packages@tensorflow.org
License: Apache 2.0
Location: c:\users\hp\anaconda3\lib\site-packages
Requires: tensorflow-intel
Required-by: 


In [36]:

mkdir test
cd test

SyntaxError: invalid syntax (327192234.py, line 2)

In [33]:
import os
import pathlib
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import PIL

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D, BatchNormalization
from tensorflow.keras import regularizers
from tensorflow.keras.callbacks import EarlyStopping,ReduceLROnPlateau,ModelCheckpoint
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam

ImportError: cannot import name 'keras' from 'tensorflow' (unknown location)

In [7]:
import splitfolders

input_folder = r'C:\Users\Hp\PlantSegmentation'
output_folder = r'C:\Users\Hp\PlantSegmentation\output'

splitfolders.ratio(input=input_folder, output=output_folder, seed=42, ratio=(.8, .0, .2))

Copying files: 0 files [00:00, ? files/s]

Copying files: 5539 files [01:54, 48.37 files/s] 


In [8]:
import glob
train_length=0
test_length=0
for name in glob.glob('/content/dataset/train/*'):
    train_length+=len(glob.glob(name+'/*.png'))

for name in glob.glob('/content/dataset/test/*'):
  test_length += len(glob.glob(name+'/*.png'))

print(train_length,test_length)

4426 1113


In [9]:
#load using keras processing
#create dataset
batch_size=32
img_height=120
img_width=120

In [10]:
data_dir_train=''
data_dir_test=''

In [18]:
!pip install tensorflow-qnd

In [20]:

train_ds=tf.keras.preprocessing.image_dataset_from_directory(data_dir_train,validation_split=0.2,subset="training",seed=123,image_size=(img_height,img_width),batch_size=batch_size)

AttributeError: module 'tensorflow' has no attribute 'keras'

In [ ]:
#creating validation dataset
val_ds=tf.keras.preprocessing.image_dataset_from_directory(data_dir_train, validation_split=0.2, subset="validation", seed=123,
                                                             image_size=(img_height, img_width), batch_size=batch_size)

In [ ]:
# Listing out all the classes of plant seedlings and store them in a list
labels=sorted(os.listdir(data_dir_train))
print(labels)

In [ ]:
#creating a list which counts the number of images in each class of plant seedling
count_plot=[]
for i in labels:
    count_plot.append(len(os.listdir(os.path.join(data_dir_train,i))))

In [ ]:
count_plot

In [ ]:
#visualizing the data
import matplotlib.pyplot as plt
count_plot=np.array(count_plot)
plt.bar(labels,count_plot)
plt.xticks(rotation=90)
plt.show()

In [ ]:
#visualizing a few images from the train data
import matplotlib.pyplot as plt
plt.figure(figsize=(10,10))
for image, label in train_ds.take(1):
    for i in range(9):
        ax=plt.subplot(3,3,i+1)
        plt.imshow(image[i].numpy().astype("uint8"))
        plt.title(labels[labels[i]])
        plt.axis("off")
    

In [ ]:
#creating the model
#specifying image dimensions
input_shape=(120,120,3)
#storing number of classes of plant seedlings in a variable
num_classes=12

#creating layers for the first model
model=Sequential()
model.add(tf.keras.layers.experimental.preprocessing.Rescaling(1./255,offset=0.0,input_shape=input_shape))
model.add(Conv2D(32,kernel_size=(3,3),input_shape=input_shape,padding='same'))
model.add(Activation('relu'))
#using batch normalization
model.add(BatchNormalization())
model.add(Conv2D(32,kernel_size=(3,3)))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, kernel_size=(3, 3), padding='same'))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(128, kernel_size=(3, 3), padding='same'))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(Conv2D(128, (3, 3)))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(256))
model.add(Activation('relu'))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

In [ ]:
#compiling the model
#choose optimizer and loss function
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metric=['accuracy'])


In [ ]:
#training the model
epochs=20
batch_size=32
history=model.fit(train_ds,validation_data=val_ds,epochs=epochs,batch_size=batch_size)


In [ ]:
#visualizing the model results
acc=history.history['accuracy']
val_acc=history.history['val_accuracy']

loss=history.history['loss']
val_loss=history.history['val_loss']

epochs_range=range(epochs)

plt.figure(figsize=(8,8))
plt.subplot(1,2,1)
plt.plot(epochs_range)
plt.plot(epochs_range,acc,label='Training Accuracy')
plt.plot(epochs_range,val_acc,label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Loss')
plt.show()